In [ ]:
import pandas as pd
import numpy as np
import os

from IPython.display import display, Math
from scipy import optimize
from math import exp, cos, sqrt, pi, sin, sqrt

from src.OspitalettoDataset import OspitalettoDataset
from src.NOAA2010Dataset import NOAA2010Dataset
from src.InsPireDataset import InsPireDataset

## Constants to be used in the notebook

In [ ]:
# The depth of the aquifer in meters. Values are around 30.
depth_aquifer = float(input("Aquifer temperature depth: "))

## Datasets reading

In [ ]:
ospitaletto2019 = OspitalettoDataset()
noaa2010Dataset = NOAA2010Dataset()
insPireDataset = InsPireDataset()

AVAILABLE_DATASETS = dict()
AVAILABLE_DATASETS.update(ospitaletto2019.load_all_data())
AVAILABLE_DATASETS.update(noaa2010Dataset.load_data())
AVAILABLE_DATASETS.update(insPireDataset.load_data())
AVAILABLE_DATASETS.keys()

In [ ]:
dataset_to_work = "stuttgart_germany"
dataset_to_work

In [ ]:
dataset = AVAILABLE_DATASETS[dataset_to_work]
dataset

# Definition of the tentative values of Tamb_avg, DT_year, d_shift_min and d_shift_max

In [ ]:
#Este codigo asume que el dataset tiene un index de tiempo.
T_hourly_avg = dataset['air_temp'].resample('H').mean().to_frame() # DataFrame de valores promedios en escala de tiempo horaria
T_hourly_avg["dayofyear"] = T_hourly_avg.index.dayofyear
T_hourly_avg["month"] = T_hourly_avg.index.month
T_hourly_avg["hourofyear"] = (T_hourly_avg.index.dayofyear - 1) * 24 + (T_hourly_avg.index.hour + 1)

T_daily_avg = dataset["air_temp"].resample('D').mean().to_frame() # DataFrame de valores promedios en escala de tiempo diaria
T_daily_avg["dayofyear"] = T_daily_avg.index.dayofyear
T_daily_avg["month"] = T_daily_avg.index.month
T_daily_avg["hourofyear"] = (T_daily_avg.index.dayofyear - 1) * 24 + (T_daily_avg.index.hour + 1)

T_ave_h = T_daily_avg.air_temp.mean() # Average dataset temperature on a daily basis
T_max = T_daily_avg.air_temp.max() # Max dataset temperature on a daily basis
DT_y = T_max - T_ave_h # Temperature swing Tmax - Tmean

d_shift_min = T_daily_avg.air_temp.idxmin(axis=0).dayofyear #Day of the min T
d_shift_max = T_daily_avg.air_temp.idxmax(axis=0).dayofyear #Day of the max T

t = T_daily_avg.dayofyear

num_hours = T_hourly_avg.shape[0] # Number of hours in a year =8737 hrs
h = pd.Series(range(1, num_hours + 1)) # A range equivalent to the hours of the year from 1 to 8761

print(f"* {T_hourly_avg=}\n"
      f"* {T_daily_avg=}\n"
      f"* {T_ave_h=}\n"
      f"* {DT_y=}\n"
      f"* {d_shift_min=}\n"
      f"* {T_daily_avg.air_temp.min()=}\n"
      f"* {d_shift_max=}\n"
      f"* {T_daily_avg.air_temp.max()=}\n"
      f"* {t=}\n"
      f"* {num_hours=}\n"
      f"* {h=}")

In [ ]:
# Verify integrity of the data
hours_of_year = T_hourly_avg.hourofyear # (T_hourly_avg.index.dayofyear - 1) * 24 + (T_hourly_avg.index.hour + 1)
hours_not_in_data = set(range(1, num_hours + 1)) - set(hours_of_year)

days_of_year = T_hourly_avg.dayofyear
days_not_in_data = set(range(1, 365 + 1)) - set(days_of_year)

months_of_year = T_hourly_avg.month
months_not_in_data = set(range(1, 12 + 1)) - set(months_of_year)

#assert len(hours_not_in_data) == 0, f"ALERT - The dataset does not contain the following hours: {hours_not_in_data}"
#assert len(days_not_in_data) == 0, f"ALERT - The dataset does not contain the following days: {days_not_in_data}"
#assert len(months_not_in_data) == 0, f"ALERT - The dataset does not contain the following months: {months_not_in_data}"

# This code finds the optimal curve that better fits the raw data through the minimum square method 

For more info visit the [scipy.optimize.curve_fit docs](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.curve_fit.html#scipy.optimize.curve_fit)

In [ ]:
def func(x, disp, amp, phi):
    t0 = len(h) # hourly resolution if x is in hours
    omega = 2 * pi / t0
    return disp + amp * np.cos(x * omega - phi) 

x_data = pd.Series(range(0,len(T_hourly_avg)))
y_data = T_hourly_avg.air_temp
params, params_covariance = optimize.curve_fit(func, x_data, y_data, p0=[T_ave_h, DT_y, d_shift_max * 24])
print(f"* {params=}\n"
      f"* {params_covariance=}")

In [ ]:
T_ave_fit = params[0]
DT_y_fit = params[1]
phi = params[2] #Corresponds to day 202

#'displacement, amplitude, and phase of the signal
print('Fitted parameters:')
display(Math(f"dist={T_ave_fit:.2f}, amp={DT_y_fit:.2f}, \\phi={phi:.2f}"))

print('Original parameters:')
display(Math(f"dist={T_ave_h:.2f}, amp={DT_y:.2f}, \\phi={d_shift_max*24:.2f}"))

# The new ambient temperature curve uses the fitted parameters as input

In [ ]:
def fitting_curve(hourofyear, T_ave_fit, DT_y_fit, T, phi, pi):
    omega = 2 * pi / T
    return T_ave_fit + DT_y_fit * np.cos(hourofyear * omega - phi) 

In [ ]:
T = T_hourly_avg.hourofyear.shape[0]
T_hourly_avg["air_temp_fit"] = T_hourly_avg.hourofyear.apply(fitting_curve, args=(T_ave_fit, DT_y_fit, T, phi, pi))

d_shift_min = T_hourly_avg.air_temp_fit.idxmin(axis=0)
d_shift_max = T_hourly_avg.air_temp_fit.idxmax(axis=0)

dd_min = T_hourly_avg[T_hourly_avg.index == d_shift_min].dayofyear.item() # d_shift_min / 24
dd_max = T_hourly_avg[T_hourly_avg.index == d_shift_max].dayofyear.item() # d_shift_max / 24

print(f'* {T=}\n'
      f'* {T_hourly_avg=}\n'
      f'* d_min (h): {d_shift_min}\n'
      f'* day_number: {dd_min}\n'
      f'* d_max (h): {d_shift_max}\n'
      f'* day_number: {dd_max}')

## Ground temperature as a function of the ambient temperature fitting curve

See for Banks [Measurements of Ground Temperature at Various Depths](https://www.researchgate.net/publication/30500372_Measurements_of_Ground_Temperature_at_Various_Depths?enrichId=rgreq-e2031a024b742c0018bb428dca3100f5-XXX&enrichSource=Y292ZXJQYWdlOzMwNTAwMzcyO0FTOjEwMTI0NTIzNDcxMjU4M0AxNDAxMTUwMTUzNjYy&el=1_x_2&_esc=publicationCoverPdf) and for Kusuda [Earth Temperatures and Thermal Diffusivity at Selected Stations in the United States](https://nvlpubs.nist.gov/nistpubs/Legacy/RPT/nbsreport8972.pdf)

In [ ]:
# The purpose of having two equations is to show that they both work using the same input data
zz = 1 # Depth [m]
alpha = 0.06048 # Ground thermal diffusivity, Banks [m^2/day]
alpha_sec = 7e-7 # Ground thermal diffusivity, Banks [m^2/s]
t_sec = 365 * 24 * 3600 # Number of seconds in a year.

t_0 = T_daily_avg.shape[0] # Number of days in a year =365 days

Tg_und = T_ave_fit # Undisturbed ground temperature
DT_y = abs(DT_y_fit)

print(f"* {zz=}\n"
      f"* {alpha=}\n"
      f"* {alpha_sec=}\n"
      f"* {t_sec=}\n"
      f"* {t_0=}\n"
      f"* {Tg_und=}\n"
      f"* {DT_y=}\n"
      f"* {dd_min=}\n"
      f"* {dd_max=}")

In [ ]:
def ground_temperature_hour(t: int):
    #t is time in hours, but the calculation is done is seconds
    T_banks= Tg_und+DT_y*exp(-zz*sqrt(pi/(alpha_sec*t_sec)))*cos(2*pi/t_sec*(t-dd_max*24)*3600-zz*sqrt(pi/(alpha_sec*t_sec))) #Banks + t_shift ->Marco's
    T_kusuda = Tg_und-DT_y*exp(-zz*sqrt(pi/(alpha_sec*t_sec)))*cos(2*pi/t_sec*((t-dd_min*24)*3600-zz/2*sqrt(t_sec/(pi*alpha_sec)))) #Kusuda
    
    return  (pd.Series({'T_banks': T_banks,'T_kusuda': T_kusuda}))

In [ ]:
tbanks_and_tkusuda = T_hourly_avg.hourofyear.apply(ground_temperature_hour)

T_hourly_avg["ground_temp"] = tbanks_and_tkusuda["T_banks"]
T_hourly_avg

## Aquifer temperature as a function of the ambient temperature fitting curve

We use the same equations as for ground temp

In [ ]:
zz = depth_aquifer

print(f"* {zz=}\n"
      f"* {alpha=}\n"
      f"* {alpha_sec=}\n"
      f"* {t_sec=}\n"
      f"* {t_0=}\n"
      f"* {Tg_und=}\n"
      f"* {DT_y=}\n"
      f"* {dd_min=}\n"
      f"* {dd_max=}")

In [ ]:
tbanks_and_tkusuda = T_hourly_avg.hourofyear.apply(ground_temperature_hour)

T_hourly_avg["aquifer_temp"] = tbanks_and_tkusuda["T_banks"]
T_hourly_avg

## Save CSV version of the dataset

In [ ]:
AVAILABLE_DATASET_PATHS = {OspitalettoDataset.OSPITALETTO: ospitaletto2019.processed_dataset_path,
                           NOAA2010Dataset.MIAMI_FL: noaa2010Dataset.processed_miami_fl_dataset_path, 
                           NOAA2010Dataset.FRESNO_CA: noaa2010Dataset.processed_fresno_ca_dataset_path, 
                           NOAA2010Dataset.OLYMPIA_WA: noaa2010Dataset.processed_olympia_wa_dataset_path,
                           NOAA2010Dataset.ROCHESTER_NY: noaa2010Dataset.processed_rochester_ny_dataset_path,
                           InsPireDataset.LONDON_UK: insPireDataset.processed_london_uk_dataset_path,
                           InsPireDataset.MADRID_SPA: insPireDataset.processed_madrid_spa_dataset_path,
                           InsPireDataset.ROME_IT: insPireDataset.processed_rome_it_dataset_path,
                           InsPireDataset.STUTTGART_GER: insPireDataset.processed_stuttgart_ger_dataset_path,}

T_hourly_avg.to_csv(path_or_buf=AVAILABLE_DATASET_PATHS[dataset_to_work])